In [153]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm

tqdm.pandas()

sparql = SPARQLWrapper("https://dbpedia.org/sparql")

In [154]:
ask_query = """
@prefix dbr:    <http://dbpedia.org/resource/> .

ASK  {{dbr:{} a owl:Thing. }}
"""

select_query1 = """
@prefix dbr:    <http://dbpedia.org/resource/> .


SELECT DISTINCT ?label WHERE {{
    {{ 
          dbr:{} <http://purl.org/linguistics/gold/hypernym> ?label. 
          ?label a owl:Thing .
    }}

}}
"""

select_query2 = """
@prefix dbr:    <http://dbpedia.org/resource/> .


SELECT DISTINCT ?label WHERE {{
    {{ 
    
          dbr:{} rdf:type ?class. 
          ?class rdfs:subClassOf* ?label .
          ?label a owl:Class.
          FILTER(?class != ?label) . 
    }}
}}
"""

In [151]:
def query_dbpedia(query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results
    
def query_item(item):
    try:
        result = query_dbpedia(ask_query.format(item))
        if result['boolean'] == 'true':
            return [f'http://dbpedia.org/resource/{item}']
        else:
            results = query_dbpedia(select_query1.format(item))
            df = pd.json_normalize(results['results']['bindings'])
            if df.shape[0] > 0:
                return df['label.value'].tolist()
            else:
                results = query_dbpedia(select_query2.format(item))
                df = pd.json_normalize(results['results']['bindings'])
                if df.shape[0] > 0:
                    return df['label.value'].tolist()
    except Exception as e:
        print(item, e)
        return []

In [62]:
def replace_characters(string):
    for ch in ["(", ")", "'", "_"]:
        string = string.replace(ch, f'\\{ch}')
    return string

In [157]:
resources = pd.read_csv('../data/dbpedia_resources.csv', index_col=False)
resources['name'] = resources['uri'].apply(lambda x: x.split('/')[-1])
resources['name'] = resources['name'].apply(lambda x: replace_characters(x))

In [158]:
resources_to_map = resources[resources['count'] >= 10]
resources_to_map

,uri,count,name
0,http://dbpedia.org/resource/White_supremacy,8774,White\_supremacy
1,http://dbpedia.org/resource/VAX,6709,VAX
2,http://dbpedia.org/resource/Hurricane_Floyd,5324,Hurricane\_Floyd
3,http://dbpedia.org/resource/Self-defense,3600,Self-defense
4,http://dbpedia.org/resource/Kenosha_Comets,3399,Kenosha\_Comets
...,...,...,...
1162,http://dbpedia.org/resource/Dam,10,Dam
1163,http://dbpedia.org/resource/Chicago_Police_Dep...,10,Chicago\_Police\_Department
1164,http://dbpedia.org/resource/God_in_Christianity,10,God\_in\_Christianity
1165,http://dbpedia.org/resource/Shorthand,10,Shorthand


In [159]:
resources_to_map['uri_list'] = resources_to_map['name'].progress_apply(query_item)

  0%|▏                                                                                | 2/1167 [00:00<04:39,  4.17it/s]

White\_supremacy HTTP Error 502: Bad Gateway


  0%|▏                                                                                | 3/1167 [00:00<04:42,  4.12it/s]

VAX HTTP Error 502: Bad Gateway


  0%|▎                                                                                | 4/1167 [00:00<04:47,  4.05it/s]

...

  5%|███▉                                                                            | 58/1167 [00:14<04:36,  4.01it/s]

Abortion HTTP Error 502: Bad Gateway


  5%|████                                                                            | 59/1167 [00:14<04:37,  4.00it/s]

QAnon HTTP Error 502: Bad Gateway


KeyboardInterrupt: 

In [ ]:
resources_to_map

In [152]:
query_item('Arizona_State_University')

['http://dbpedia.org/resource/University']

In [131]:
resources_to_map.set_index('uri')[['uri_list']].explode('uri_list').to_csv('../data/dbpedia_resource_mapping.csv')